In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
file_path = "/content/Stress.csv"
df = pd.read_csv(file_path)

In [ ]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [ ]:
# text preprocessing
df['clean_text'] = df['text'].astype(str).apply(clean_text)

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=5000)  # uses the top 5000 words
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_len = max(len(seq) for seq in X_train_seq)  # Find max length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [ ]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_len),  # embedding layer
    LSTM(128, return_sequences=True),  # LSTM layer
    Dropout(0.3),
    LSTM(64),  # another LSTM layer
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # output layer (binary)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 62s 871ms/step - accuracy: 0.5203 - loss: 0.6923 - val_accuracy: 0.5370 - val_loss: 0.6909
Epoch 2/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 53s 744ms/step - accuracy: 0.5274 - loss: 0.6915 - val_accuracy: 0.5370 - val_loss: 0.6909
Epoch 3/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 53s 749ms/step - accuracy: 0.5363 - loss: 0.6919 - val_accuracy: 0.5370 - val_loss: 0.6914
Epoch 4/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 83s 767ms/step - accuracy: 0.5121 - loss: 0.6930 - val_accuracy: 0.5370 - val_loss: 0.6910
Epoch 5/5
71/71 ━━━━━━━━━━━━━━━━━━━━ 81s 749ms/step - accuracy: 0.5315 - loss: 0.6921 - val_accuracy: 0.5370 - val_loss: 0.6915


In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 0.5440 - loss: 0.6911
Test Accuracy: 0.54


In [ ]:
def predict_stress(sentence):
    sentence = clean_text(sentence)
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_seq = pad_sequences(sequence, maxlen=max_len, padding='post')
    prediction = model.predict(padded_seq)[0][0]
    return "Stressful" if prediction > 0.5 else "Not Stressful"

In [ ]:
new_sentence = "I can’t sleep at night and my mind feels constantly overwhelmed."
print(f"Prediction: {predict_stress(new_sentence)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Prediction: Stressful
